Import arcpy, ustawienie środowiska projektu

In [1]:
import arcpy
from arcpy import *

arcpy.env.workspace = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdp")
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS 1989 Poland CS92")
arcpy.env.overwriteOutput = True
arcpy.env.extent = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\swieradow_zdroj_Buffer")
arcpy.env.mask = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\swieradow_zdroj_Buffer.shp")
arcpy.env.cellSize = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\raster_extracted.tif")
gmina_mask = arcpy.env.mask
gmina_extent = arcpy.env.extent

aprx = arcpy.mp.ArcGISProject("CURRENT")  # Otwiera bieżący projekt
m = aprx.activeMap

# wylaczenie dodawania warstw
arcpy.env.addOutputsToMap = False

Wczytanie warstw i ich zapis do zmiennych

In [2]:
drogi_wszystkie = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\SKDR.shp")
rzeki = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\SWRS.shp")
jeziora = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\PTWP.shp")
lasy = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\PTLZ.shp")
budynki_wszystkie = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\BUBDA.shp")
bufor = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\swieradow_zdroj_Buffer")
raster_gmina = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\raster_extracted")

Selekcja po atrybutach odpowiednich warstw

In [3]:
#wyselekcjonowanie drog utwardzonych
drogi = arcpy.management.SelectLayerByAttribute(drogi_wszystkie, "NEW_SELECTION", "MATE_NAWIE IN ('beton','bruk', 'kostka kamienna', 'kostka prefabrykowana', 'masa bitumiczna', 'płyty betonowe')", None)
arcpy.management.CopyFeatures(drogi, r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\drogi_selected", '', None, None, None)
drogi_selected = 'drogi_selected'
arcpy.SelectLayerByAttribute_management(drogi, "CLEAR_SELECTION")

#wyselekcjonowanie budynkow mieszkalnych
budynki = arcpy.management.SelectLayerByAttribute(budynki_wszystkie, "NEW_SELECTION", "FOBUD IN ('budynki mieszkalne')", None)
arcpy.management.CopyFeatures(budynki, r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\budynki_selected", '', None, None, None)
budynki_selected = 'budynki_selected'
arcpy.SelectLayerByAttribute_management(budynki, "CLEAR_SELECTION")

<Result 'BUBDA_Layer1'>

Stworzenie map odległości dla kryteriów narzędziem Euclidian Distance

In [4]:
distance_rzeki = arcpy.sa.EucDistance(rzeki, None, "5")
distance_jeziora = arcpy.sa.EucDistance(jeziora, None, "5")
distance_drogi = arcpy.sa.EucDistance(drogi_selected, None, "5")
distance_budynki = arcpy.sa.EucDistance(budynki_selected, None, "5")
distance_lasy = arcpy.sa.EucDistance(lasy, None, "5")

# mapa odleglosci do kryterium z wezlami wykonana w QGiS:
distance_wezly  = Raster(r"C:\Users\mplaciszewska\Desktop\studia\geoinf_sem_5\analizy_przestrzenne\cw1\kryterium_wezly_komunikacyjne.tif")

Kryterium 1 - odległość od rzek i zbiorników wodnych, jak najbliżej (nieprzekraczalna strefa ochronna 100m)

In [5]:
#kryterium ostre dla strefy ochronnej 100m poprzez reklasyfikacje map odleglosci
rzeki_ostre = arcpy.ddd.Reclassify(distance_rzeki, "VALUE", "0 100 0;100 912,537659 1")
jeziora_ostre = arcpy.ddd.Reclassify(distance_jeziora, "VALUE", "0 100 0;100 1773,760132 1")

#połączenie kryteriów ostrych dla rzek i jezior
wody_ostre = arcpy.sa.FuzzyOverlay([rzeki_ostre, jeziora_ostre], "AND")

In [6]:
#kryterium rozmyte dla obszarów w odległości powyżej 100m
rzeki_rozmyte = arcpy.sa.FuzzyMembership(distance_rzeki, arcpy.sa.FuzzyLinear(400, 200))
jeziora_rozmyte = arcpy.sa.FuzzyMembership(distance_jeziora, arcpy.sa.FuzzyLinear(400, 200))

#połączenie kryteriów rozmytych dla rzek i jezior
wody_rozmyte = arcpy.sa.FuzzyOverlay([rzeki_rozmyte, jeziora_rozmyte], "OR")

In [7]:
#połączenie kryterium ostrego i rozmytego dla wody
kryterium_wody = arcpy.sa.FuzzyOverlay([wody_ostre, wody_rozmyte], 'AND')
kryterium_wody.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_wody")

Kryterium 2 - dostęp do dróg utwardzonych, jak największe zagęszczenie 

In [8]:
# ocena przydatnosci pod wzgledem zageszczenia
raster_zageszczenia = arcpy.sa.LineDensity(
    in_polyline_features=drogi_selected,
    population_field="NONE",
    cell_size=5,
    search_radius=1000,
    area_unit_scale_factor="SQUARE_METERS"
)

max_value = raster_zageszczenia.maximum

# przeskalowanie rastra zageszczenia do wartosci 0-1
kryterium_drogi = arcpy.sa.RescaleByFunction(
    in_raster=raster_zageszczenia,
    transformation_function= f"LINEAR 0 1,{max_value} 0 # {max_value} #",
    from_scale=0,
    to_scale=1
)

kryterium_drogi.save(r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_drogi")
m.addDataFromPath(r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_drogi")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000860: Input polyline features: is not the type of Composite Geodataset, or does not exist.
Failed to execute (LineDensity).


Kryterium 3 - odległość od budynków mieszkalnych

In [ ]:
kryterium_budynki = arcpy.sa.FuzzyMembership(distance_budynki,  arcpy.sa.FuzzyLinear(150, 400))
kryterium_budynki.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_budynki")

Krterium 4 - pokrycie terenu, nie w lesie, powyżej 15m od lasu, optymalnie 100m od lasu 

In [ ]:
kryterium_lasy = arcpy.sa.FuzzyMembership(distance_lasy, arcpy.sa.FuzzyLinear(15, 100))
kryterium_lasy.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_lasy")

Kryterium 5 - nachylenie stoków, optymalnie płasko, maksymalnie 10%

In [ ]:
#stworzenie warstwy z nachyleniami stoków narzędziem Slope
spadki_output = R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\spadki"
spadki = arcpy.ddd.Slope(raster_gmina, spadki_output, "PERCENT_RISE", 1)

kryterium_spadki = arcpy.sa.FuzzyMembership(spadki, arcpy.sa.FuzzyLinear(10, 3))
kryterium_spadki.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_spadki")

Kryterium 6 - nasłonecznienie, optymalnie stoki południowe (SW-SE)

In [ ]:
#stworzenie warstwy ze stokami narzędziem Aspect
stoki_output = R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\stoki"
stoki = arcpy.ddd.Aspect(raster_gmina, stoki_output)

# rozdzielenie kryterium rozmytego na rosnący i malejący
stoki_ES = arcpy.sa.FuzzyMembership(stoki, arcpy.sa.FuzzyLinear(90, 113))
stoki_WS = arcpy.sa.FuzzyMembership(stoki, arcpy.sa.FuzzyLinear(270, 248))

# ustawienie przydatności płaskich stoków na 1
plaskie_stoki = arcpy.sa.Con(stoki, 1, 0, "VALUE = -1") 

stoki_rozmyte = arcpy.sa.FuzzyOverlay([stoki_ES, stoki_WS], 'AND')
kryterium_stoki = arcpy.sa.FuzzyOverlay([stoki_rozmyte, plaskie_stoki], 'OR')

kryterium_stoki.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_stoki")

Utworzenie węzłów komunikacyjnych

In [ ]:
# # tymczasowe wyłączenie maski
# arcpy.env.mask = None
# arcpy.env.extent = None

# # wczytanie warstwy drog dla wojewodztwa
# drogi_wojewodztwa = "SKDR_wojewodztwo.shp"
# # selekcja dróg o odpowiedniej kategorii
# drogi_wezlowe = arcpy.management.SelectLayerByAttribute(drogi_wszystkie, "NEW_SELECTION", "KAT_ZARZAD IN ('wojewódzka','powiatowa')", None)
# arcpy.management.CopyFeatures(drogi_wezlowe, r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\drogi_wezlowe_selected", '', None, None, None)
# drogi_wezlowe_selected = 'drogi_wezlowe_selected'
# arcpy.SelectLayerByAttribute_management(drogi_wezlowe, "CLEAR_SELECTION")
# m.addDataFromPath(r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\drogi_wezlowe_selected")
# # utworzenie punktow przeciecia drog
# punkty_przeciecia_drog = arcpy.analysis.Intersect(drogi_wezlowe_selected,
#     out_feature_class = None,
#     join_attributes="ALL",
#     cluster_tolerance=None,
#     output_type="POINT"
# )

# # spatial join do policzenia liczby polaczen i selekcja wezlow o liczbie polaczen > 2
# punkty_przeciec_polaczenia = arcpy.analysis.SpatialJoin(
#     target_features=punkty_przeciecia_drog,
#     join_features=drogi_wezlowe_selected,
#     out_feature_class=None,
#     join_operation="JOIN_ONE_TO_ONE",
#     join_type="KEEP_ALL",
#     field_mapping='FID_drogi_wezlowe_selected "FID_drogi_wezlowe_selected" true true false 4 Long 0 0,First,#,wezly_komunikacyjne,FID_drogi_wezlowe_selected,-1,-1;TERYT "TERYT" true true false 4 Text 0 0,First,#,wezly_komunikacyjne,TERYT,0,4;LOKALNYID "LOKALNYID" true true false 50 Text 0 0,First,#,wezly_komunikacyjne,LOKALNYID,0,50;PRZES_NAZW "PRZES_NAZW" true true false 25 Text 0 0,First,#,wezly_komunikacyjne,PRZES_NAZW,0,25;WERSJA "WERSJA" true true false 21 Text 0 0,First,#,wezly_komunikacyjne,WERSJA,0,21;POCZ_WERSJ "POCZ_WERSJ" true true false 21 Text 0 0,First,#,wezly_komunikacyjne,POCZ_WERSJ,0,21;OZNA_ZMIAN "OZNA_ZMIAN" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,OZNA_ZMIAN,0,254;ZRO_DANYCH "ZRO_DANYCH" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,ZRO_DANYCH,0,254;KAT_ISTNIE "KAT_ISTNIE" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,KAT_ISTNIE,0,254;UWAGI "UWAGI" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,UWAGI,0,254;INFO_DODAT "INFO_DODAT" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,INFO_DODAT,0,254;KOD10K "KOD10K" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,KOD10K,0,254;SKROT_KART "SKROT_KART" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,SKROT_KART,0,254;KAT_ZARZAD "KAT_ZARZAD" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,KAT_ZARZAD,0,254;KLASA_DROG "KLASA_DROG" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,KLASA_DROG,0,254;MATE_NAWIE "MATE_NAWIE" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,MATE_NAWIE,0,254;NAZWA_DROG "NAZWA_DROG" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,NAZWA_DROG,0,254;LICZ_JEZDN "LICZ_JEZDN" true true false 8 Double 0 0,First,#,wezly_komunikacyjne,LICZ_JEZDN,-1,-1;POLOZENIE "POLOZENIE" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,POLOZENIE,0,254;SZER_NAWIE "SZER_NAWIE" true true false 8 Double 0 0,First,#,wezly_komunikacyjne,SZER_NAWIE,-1,-1;NUMER_DROG "NUMER_DROG" true true false 254 Text 0 0,First,#,wezly_komunikacyjne,NUMER_DROG,0,254;TERYT_1 "TERYT" true true false 4 Text 0 0,First,#,drogi_wezlowe_selected,TERYT,0,4;LOKALNYID_1 "LOKALNYID" true true false 50 Text 0 0,First,#,drogi_wezlowe_selected,LOKALNYID,0,50;PRZES_NAZW_1 "PRZES_NAZW" true true false 25 Text 0 0,First,#,drogi_wezlowe_selected,PRZES_NAZW,0,25;WERSJA_1 "WERSJA" true true false 21 Text 0 0,First,#,drogi_wezlowe_selected,WERSJA,0,21;POCZ_WERSJ_1 "POCZ_WERSJ" true true false 21 Text 0 0,First,#,drogi_wezlowe_selected,POCZ_WERSJ,0,21;OZNA_ZMIAN_1 "OZNA_ZMIAN" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,OZNA_ZMIAN,0,254;ZRO_DANYCH_1 "ZRO_DANYCH" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,ZRO_DANYCH,0,254;KAT_ISTNIE_1 "KAT_ISTNIE" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,KAT_ISTNIE,0,254;UWAGI_1 "UWAGI" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,UWAGI,0,254;INFO_DODAT_1 "INFO_DODAT" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,INFO_DODAT,0,254;KOD10K_1 "KOD10K" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,KOD10K,0,254;SKROT_KART_1 "SKROT_KART" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,SKROT_KART,0,254;KAT_ZARZAD_1 "KAT_ZARZAD" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,KAT_ZARZAD,0,254;KLASA_DROG_1 "KLASA_DROG" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,KLASA_DROG,0,254;MATE_NAWIE_1 "MATE_NAWIE" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,MATE_NAWIE,0,254;NAZWA_DROG_1 "NAZWA_DROG" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,NAZWA_DROG,0,254;LICZ_JEZDN_1 "LICZ_JEZDN" true true false 8 Double 0 0,First,#,drogi_wezlowe_selected,LICZ_JEZDN,-1,-1;POLOZENIE_1 "POLOZENIE" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,POLOZENIE,0,254;SZER_NAWIE_1 "SZER_NAWIE" true true false 8 Double 0 0,First,#,drogi_wezlowe_selected,SZER_NAWIE,-1,-1;NUMER_DROG_1 "NUMER_DROG" true true false 254 Text 0 0,First,#,drogi_wezlowe_selected,NUMER_DROG,0,254;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,drogi_wezlowe_selected,Shape_Length,-1,-1',
#     match_option="INTERSECT",
#     search_radius=None,
#     distance_field_name=""
# )
# # Wyrażenie wyboru
# zapytanie = '"Join_Count" > 2'

# # Selekcja punktów
# wezly_komunikacyjne = arcpy.management.SelectLayerByAttribute(
#     punkty_przeciec_polaczenia,  # warstwa z policzonymi połączeniami
#     "NEW_SELECTION",
#     zapytanie
# )
# wezly_komunikacyjne_layer = r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\wezly_komunikacyjne"
# arcpy.management.CopyFeatures(wezly_komunikacyjne, wezly_komunikacyjne_layer, '', None, None, None)
# wezly_komunikacyjne_selected = 'wezly_komunikacyjne_selected'
# arcpy.SelectLayerByAttribute_management(wezly_komunikacyjne, "CLEAR_SELECTION")

# m.addDataFromPath(wezly_komunikacyjne_layer)

# # przywrócenie maski
# arcpy.env.mask = gmina_mask
# arcpy.env.extent = gmina_extent

kryterium_wezly = arcpy.sa.FuzzyMembership(distance_wezly, arcpy.sa.FuzzyLinear(distance_wezly.maximum, 200))
kryterium_wezly.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_wezly")
m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_wezly")

Wczytanie działek